# LSTM on Amazon fine food review

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer as sno
setofstopwords=set(stopwords.words('english'))
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\Ankit Kumar
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Loading from database

In [2]:
conn= sqlite3.connect('database.sqlite')
data= pd.read_sql_query('''
SELECT * FROM Reviews WHERE Score!=3
''',conn)
data.shape

(525814, 10)

### Removing not helpful reviews

In [3]:
data['Score']=data['Score'].map(lambda x:'Positive' if x>3 else 'Negative')
sorteddata= data.sort_values('ProductId',axis=0)
finaldata= sorteddata.drop_duplicates(subset={'UserId','ProfileName',\
        'Time','Text'}, keep='first',inplace=False)

finaldata= finaldata[finaldata['HelpfulnessNumerator'] <= finaldata['HelpfulnessDenominator']]
data= finaldata.sort_values('Time',axis=0)
data.shape

(364171, 10)

### Cleaning HTML, punctuations, apply stemming, lowercasing etc without removing stopwords

In [4]:
def cleanhtml(sentance): #substitute expression contained in <> with ' '
    cleaned= re.sub(re.compile('<.*?>'),' ',sentance)
    return cleaned
#function for removing punctuations chars
def cleanpunc(sentance):
    cleaned= re.sub(r'[?|!|\'|"|#]',r'',sentance)
    cleaned= re.sub(r'[.|,|)|(|\|/]',r'',sentance)
    return cleaned
snowstem= sno('english')

i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
for sent in data['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        # we have used cleanpunc(w).split(), one more split function here 
        # because consider w="abc.def", cleanpunc(w) will return "abc def"
        # if we dont use .split() function then we will be considring "abc def" 
        # as a single word, but if you use .split() function we will get "abc", "def"
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                s=(snowstem.stem(cleaned_words.lower())).encode('utf8')
                filtered_sentence.append(s)
                if(data['Score'].values)[i] =='Positive':
                    all_positive_words.append(s)
                if(data['Score'].values)[i] =='Negative':
                    all_negative_words.append(s)
            else:
                continue
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    final_string.append(str1)

# storing data till now
data['CleanedText']=final_string 
#adding a column of CleanedText which displays the data after pre-processing of the review 
data['CleanedText']=data['CleanedText'].str.decode("utf-8")
    # store final table into an SQlLite table for future.
conn = sqlite3.connect('cleanedTextData.sqlite')
c=conn.cursor()
conn.text_factory = str
data.to_sql('Reviews', conn,  schema=None, if_exists='replace', \
        index=True, index_label=None, chunksize=None, dtype=None)
conn.close()

In [5]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,Positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,this witti littl book make son laugh loud reci...
138683,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,Positive,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,can rememb see the show when air televis year ...
417839,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,Positive,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,beetlejuic well written movi everyth about fro...
346055,374359,B00004CI84,A344SMIA5JECGM,Vincent P. Ross,1,2,Positive,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",twist rumplestiskin captur film star michael k...
417838,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,Positive,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,beetlejuic excel and funni movi keaton hilari ...


In [7]:
data['Text'][2]

'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.'

In [8]:
data['CleanedText'][2]

'this confect that has been around few centuri light pillowi citrus gelatin with nut this case filbert and cut into tini squar and then liber coat with powder sugar and tini mouth heaven not too chewi and veri flavor high recommend this yummi treat you are familiar with the stori lion the witch and the this the treat that seduc edmund into sell out his brother and sister the witch'

### Taking 100k datapoints

In [20]:
Data= data[:100000]
Data= Data[['CleanedText','Score']]
Data['Score']= Data['Score'].map(lambda x:1 if x=='Positive' else 0)

Data_x= Data['CleanedText']
Data_y= Data['Score']

In [82]:
Data_x.index= [i for i in range(0,10**5)]
Data_x

0        this witti littl book make son laugh loud reci...
1        can rememb see the show when air televis year ...
2        beetlejuic well written movi everyth about fro...
3        twist rumplestiskin captur film star michael k...
4        beetlejuic excel and funni movi keaton hilari ...
5        this one movi that should your movi collect fi...
6        myself alway enjoy this movi veri funni and en...
7        bought few these after apart was infest with f...
8        what happen when you say his name three michae...
9        get look for beatlejuic french version video r...
10       get crazi realli imposs today not find the fre...
11       this was realli good idea and the final produc...
12       just receiv shipment and could hard wait tri t...
13       have just recent purchas the woodstream corp g...
14       this are much easier use than the wilson past ...
15       these are easi use they not make mess and offe...
16       this such great film even know how sum first a.

### Making Vocabulary set and Frequency dictionary of words

In [45]:
# collecting all words in single list
list_= []
for i in Data_x:
    list_ += i
list_= ''.join(list_)
allWords=list_.split()

In [56]:
vocabulary= set(allWords)

In [60]:
vocabulary_list= list(vocabulary)

In [62]:
#frequency dictionary
freq_dict= {}
for word in vocabulary_list:
    freq_dict[word]= allWords.count(word)

In [64]:
freq_dict

{'weightthis': 7,
 'againdentist': 1,
 'milibar': 1,
 'almondopen': 1,
 'needuse': 2,
 'getcha': 1,
 'casealthough': 1,
 'excelhusband': 3,
 'glycemix': 1,
 'pricesuch': 1,
 'tastestri': 1,
 'enforc': 4,
 'adamhave': 1,
 'eitherheard': 1,
 'gravel': 16,
 'fluffihave': 1,
 'giftmake': 2,
 'futil': 3,
 'ratewhen': 1,
 'statego': 1,
 'dealorvill': 1,
 'mcdougalorder': 1,
 'art': 123,
 'coppicino': 1,
 'intensmuch': 1,
 'turkeynot': 1,
 'prais': 116,
 'caseshare': 1,
 'lovenormal': 3,
 'saltlive': 1,
 'railway': 1,
 'othercrazi': 1,
 'wazu': 1,
 'weekcent': 1,
 'coloraft': 1,
 'asthamt': 1,
 'adbig': 1,
 'outragtri': 1,
 'thougheveryon': 1,
 'owenlike': 1,
 'stravecchio': 1,
 'eatcultiv': 1,
 'believthe': 3,
 'cartthis': 1,
 'issuesat': 1,
 'theobromin': 9,
 'bussi': 4,
 'simiar': 2,
 'immediwas': 1,
 'promishow': 1,
 'detoxifiuse': 1,
 'foamread': 1,
 'breadmix': 3,
 'thesedoctor': 1,
 'productcoffe': 1,
 'chickenan': 1,
 'getorder': 7,
 'morecavali': 1,
 'youunfroast': 1,
 'alikkid': 1,


In [66]:
import pickle
with open('freq_dict.pkl','wb') as file:
    pickle.dump(freq_dict,file)

### Creating rank list of frequent words upto 5000

In [73]:
from operator import itemgetter
sorted_list= []
for k, v in sorted(freq_dict.items(), key=itemgetter(1),reverse=True):
    sorted_list.append(k)

In [74]:
sorted_list

['the',
 'and',
 'this',
 'for',
 'that',
 'with',
 'you',
 'have',
 'but',
 'are',
 'not',
 'they',
 'was',
 'like',
 'tast',
 'flavor',
 'them',
 'these',
 'good',
 'tea',
 'one',
 'use',
 'can',
 'product',
 'veri',
 'great',
 'just',
 'tri',
 'all',
 'from',
 'love',
 'make',
 'has',
 'when',
 'get',
 'more',
 'other',
 'will',
 'than',
 'coffe',
 'had',
 'out',
 'would',
 'some',
 'buy',
 'food',
 'onli',
 'eat',
 'about',
 'time',
 'your',
 'find',
 'realli',
 'also',
 'best',
 'much',
 'too',
 'littl',
 'order',
 'even',
 'amazon',
 'becaus',
 'drink',
 'which',
 'were',
 'price',
 'bag',
 'there',
 'store',
 'been',
 'mix',
 'what',
 'chocol',
 'ani',
 'better',
 'well',
 'box',
 'sugar',
 'now',
 'year',
 'their',
 'after',
 'sweet',
 'found',
 'day',
 'dog',
 'want',
 'then',
 'high',
 'look',
 'our',
 'give',
 'cup',
 'over',
 'first',
 'add',
 'water',
 'brand',
 'recommend',
 'most',
 'she',
 'made',
 'think',
 'packag',
 'way',
 'who',
 'treat',
 'two',
 'nice',
 'work',


In [83]:
Data_x[1]

'can rememb see the show when air televis year ago when was child sister later bought the which have this day thirti somethingi use this seri book song when did student teach for preschool turn the whole school now purchas along with the book for children the tradit live'

In [91]:
top_words= 5000
sorted_list= sorted_list[:5000]

### Transforming Sentences of words to sequence of rank number of words

In [92]:
column=[]
for sent in Data_x:
    lis=[]
    for word in sent.split():
        if word in sorted_list:
            lis.append(word)
    column.append(' '.join(lis))

In [94]:
with open('column.pkl','wb') as file:
    pickle.dump(column,file)

In [97]:
final_x=[]
for sent in Data_x:
    lis=[]
    for word in sent.split():
        if word in sorted_list:
            lis.append(sorted_list.index(word)+1)
    final_x.append(lis)

In [99]:
Xtest= final_x[:30000]
Ytest= Data_y[:30000]
Xtrain= final_x[30000:]
Ytrain= Data_y[30000:]

In [105]:
print(Xtrain[1])

[8, 216, 209, 1980, 106, 8, 368, 25, 3960, 528, 3217, 3217, 721, 17, 488, 1101, 2, 583, 127, 5, 1071, 474, 5, 11, 697, 8, 47, 28, 1, 557, 16, 3217, 3217, 9, 322, 1, 31, 89, 99, 12, 10, 156, 622, 42, 91, 151, 69, 541, 35, 1134, 17, 61, 5, 8, 352, 1, 69]


## Applying LSTM models

In [106]:
from keras.preprocessing import sequence
max_review_length=600
Xtrain = sequence.pad_sequences(Xtrain, maxlen=max_review_length)
Xtest= sequence.pad_sequences(Xtest, maxlen=max_review_length)

Using TensorFlow backend.


In [108]:
print(Xtrain[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [114]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9021134197561777072
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3206820659
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17023240268385776602
physical_device_desc: "device: 0, name: GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [115]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

### Model 1 same as of IMDB

In [112]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 32)           160032    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,333
Trainable params: 213,333
Non-trainable params: 0
_________________________________________________________________
None


In [121]:
history=model.fit(Xtrain, Ytrain, nb_epoch=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(Xtest, Ytest, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Instructions for updating:
Use tf.cast instead.


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/10
70000/70000 [==============================] - 1607s 23ms/step - loss: 0.2394 - acc: 0.9088
Epoch 2/10
70000/70000 [==============================] - 1620s 23ms/step - loss: 0.1763 - acc: 0.9321
Epoch 3/10
70000/70000 [==============================] - 1715s 24ms/step - loss: 0.1579 - acc: 0.9392
Epoch 4/10
70000/70000 [==============================] - 1468s 21ms/step - loss: 0.1422 - acc: 0.9458
Epoch 5/10
70000/70000 [==============================] - 1459s 21ms/step - loss: 0.1260 - acc: 0.9525
Epoch 6/10
70000/70000 [==============================] - 1518s 22ms/step - loss: 0.1114 - acc: 0.9583
Epoch 7/10
70000/70000 [==============================] - 1522s 22ms/step - loss: 0.0993 - acc: 0.9631
Epoch 8/10
70000/70000 [==============================] - 1554s 22ms/step - loss: 0.0873 - acc: 0.9674
Epoch 9/10
70000/70000 [==============================] - 1459s 21ms/step - loss: 0.0784 - acc: 0.9716
Epoch 10/10
70000/70000 [==============================] - 1445s 21ms/ste

### Model 2 

In [120]:
# create the model
embedding_vecor_length = 32
model2 = Sequential()
model2.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model2.add(LSTM(100,return_sequences=True))
model2.add(Dropout(0.25))
model2.add(LSTM(80))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 600, 32)           160032    
_________________________________________________________________
lstm_9 (LSTM)                (None, 600, 100)          53200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 600, 100)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 80)                57920     
_________________________________________________________________
dropout_5 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 81        
Total params: 271,233
Trainable params: 271,233
Non-trainable params: 0
_________________________________________________________________
None

In [122]:
history2= model2.fit(Xtrain, Ytrain, nb_epoch=10, batch_size=64)-# Final evaluation of the model
scores = model2.evaluate(Xtest, Ytest, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/10
70000/70000 [==============================] - 2961s 42ms/step - loss: 0.2402 - acc: 0.9088
Epoch 2/10
70000/70000 [==============================] - 2958s 42ms/step - loss: 0.1802 - acc: 0.9316
Epoch 3/10
70000/70000 [==============================] - 2961s 42ms/step - loss: 0.1613 - acc: 0.9386
Epoch 4/10
70000/70000 [==============================] - 2958s 42ms/step - loss: 0.1429 - acc: 0.9459
Epoch 5/10
70000/70000 [==============================] - 2964s 42ms/step - loss: 0.1254 - acc: 0.9528
Epoch 6/10
70000/70000 [==============================] - 3007s 43ms/step - loss: 0.1108 - acc: 0.9592
Epoch 7/10
70000/70000 [==============================] - 2996s 43ms/step - loss: 0.0970 - acc: 0.9653
Epoch 8/10
70000/70000 [==============================] - 3001s 43ms/step - loss: 0.0839 - acc: 0.9706
Epoch 9/10
70000/70000 [==============================] - 3001s 43ms/step - loss: 0.0764 - acc: 0.9736
Epoch 10/10
70000/70000 [==============================] - 3011s 43ms/ste

In [123]:
with open('model.pkl','wb') as file:
    pickle.dump(model,file)
with open('model2.pkl','wb') as file:
    pickle.dump(model2,file)

## Testing our model on self made review sentence

In [165]:
# making a function which convert sentance to requred vectorized format that will feed well in model

def cleanhtml(sentance): #substitute expression contained in <> with ' '
    cleaned= re.sub(re.compile('<.*?>'),' ',sentance)
    return cleaned
#function for removing punctuations chars
def cleanpunc(sentance):
    cleaned= re.sub(r'[?|!|\'|"|#]',r'',sentance)
    cleaned= re.sub(r'[.|,|)|(|\|/]',r'',sentance)
    return cleaned
snowstem= sno('english')

def predict_this(sentance):
    i=0
    str1=' '
    final_string=[]
    all_positive_words=[] # store words from +ve reviews here
    all_negative_words=[] # store words from -ve reviews here.
    sent= sentance
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        # we have used cleanpunc(w).split(), one more split function here 
        # because consider w="abc.def", cleanpunc(w) will return "abc def"
        # if we dont use .split() function then we will be considring "abc def" 
        # as a single word, but if you use .split() function we will get "abc", "def"
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                s=(snowstem.stem(cleaned_words.lower())).encode('utf8')
                filtered_sentence.append(s)
                if(data['Score'].values)[i] =='Positive':
                    all_positive_words.append(s)
                if(data['Score'].values)[i] =='Negative':
                    all_negative_words.append(s)
            else:
                continue
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    final_string.append(str1)

    final_string
    for i in final_string:
        final_string=i.decode("utf-8")

    lis=[]
    for word in final_string.split():
        if word in sorted_list:
            lis.append(sorted_list.index(word)+1)

    final_string= lis
    final_string = sequence.pad_sequences([final_string], maxlen=max_review_length)
    print(final_string)
    what= ''
    if (round(float(model2.predict(final_string)))==1):
        what= 'Positive'
        acc= round(float(model2.predict(final_string))*100,2)
    else:
        what='Negative'
        acc= 100- round(float(model2.predict(final_string))*100,2)
    print(what,'review with',acc,'% Accuracy')

In [174]:
sentance= 'food was very bad in taste'
predict_this(sentance)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 

In [175]:
sentance= 'taste of chocolate was fantastic'
predict_this(sentance)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 

In [176]:
sentance= 'food was medium tasty'
predict_this(sentance)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 

In [179]:
print(data['Text'][1])
predict_this(data['Text'][1])

Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 

# Summary 

* Accuracy of First model is 92.31%
* Accuracy of Second model is 92.96%